In [35]:
import pandas as pd
import json

from ib_insync import *

util.startLoop()
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

market = 'snp'
# ...variables initialization
with open('var.json', 'r') as fp:
    data = json.load(fp)

host = data['common']['host']
port = data[market]['port']
cid = 1

symbol = 'FDX'

with IB().connect(host=host, port=port, clientId=cid) as ib:
    c = ib.qualifyContracts(Stock(symbol, 'SMART', 'USD'))[0]
    ohlc = util.df(ib.reqHistoricalData(contract=c, endDateTime = '',
                               durationStr='365 D', barSizeSetting= '1 day',
                               whatToShow = 'Trades', useRTH=True))

In [42]:
# reverse sort to have latest date on top
ohlc.sort_values(by='date', inplace=True, ascending=False)
ohlc.head(8)

,date,open,high,low,close,volume,average,barCount
364,2019-12-02,160.46,163.70,158.03,158.03,17906,159.9820,11865
363,2019-11-29,161.11,161.46,159.83,160.05,4912,160.5250,3064
362,2019-11-27,161.75,162.07,160.21,161.51,11305,161.3025,7054
361,2019-11-26,159.71,162.76,158.63,161.50,17314,161.5535,12333
360,2019-11-25,157.41,160.34,156.39,159.54,14683,159.2205,9508
359,2019-11-22,152.31,156.79,152.26,156.55,14543,154.9545,9598
358,2019-11-21,152.50,153.77,151.74,152.08,10366,152.4525,6273
357,2019-11-20,155.00,155.09,151.55,152.27,14816,152.8840,9425


In [ ]:
ohlc.high.expanding(1).max() -ohlc.low.expanding(1).min()

In [ ]:
ohlc.high.expanding(1).min() -ohlc.low.expanding(1).min()

In [24]:
import pandas as pd
import numpy as np
df = pd.DataFrame(np.random.randint(1000, size=100), index=range(100), columns = ['reading'])

In [25]:
df.reading.rolling(
    

,reading
0,888
1,90
2,118
3,685
4,91
...,...
95,860
96,349
97,326
98,311


In [16]:
def risefall(df, i):
    j = len(df) - i
    df1 = df.assign(delta = df.hi.rolling(j).max() - df.lo.rolling(j).min(), 
                        pctchange = df.avg.pct_change(periods=j))
    max_fall = df1[df1.pctchange<=0].delta.max()
    max_rise = df1[df1.pctchange>0].delta.max()
    return max_rise, max_fall

In [19]:
df.head()

,hi,lo,avg
0,363,340,351.5
1,464,384,424.0
2,376,297,336.5
3,676,626,651.0
4,855,765,810.0


In [21]:
[risefall(df, k) for k, v in df.iterrows()]

[(nan, nan),
 (1049.0, nan),
 (1049.0, nan),
 (1049.0, nan),
 (1049.0, 1049.0),
 (1049.0, 1049.0),
 (1049.0, 1049.0),
 (1049.0, 1049.0),
 (1049.0, 1049.0),
 (1049.0, 1049.0),
 (1049.0, 1049.0),
 (1049.0, 1049.0),
 (1049.0, 1049.0),
 (1049.0, 1049.0),
 (1049.0, 1049.0),
 (1049.0, 1049.0),
 (1049.0, 1049.0),
 (1049.0, 1049.0),
 (1049.0, 1049.0),
 (1049.0, 1049.0),
 (1049.0, 1049.0),
 (1049.0, 1049.0),
 (1049.0, 1049.0),
 (1049.0, 1049.0),
 (1049.0, 1049.0),
 (1049.0, 1049.0),
 (1049.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0, 1032.0),
 (1032.0

In [22]:
pd.__version__

'0.25.2'

In [23]:
pd.show_versions()


INSTALLED VERSIONS
------------------
commit           : None
python           : 3.8.0.final.0
python-bits      : 32
OS               : Windows
OS-release       : 10
machine          : AMD64
processor        : Intel64 Family 6 Model 142 Stepping 9, GenuineIntel
byteorder        : little
LC_ALL           : None
LANG             : None
LOCALE           : English_United States.1252

pandas           : 0.25.2
numpy            : 1.17.3
pytz             : 2019.3
dateutil         : 2.8.0
pip              : 19.3.1
setuptools       : 41.2.0
Cython           : None
pytest           : None
hypothesis       : None
sphinx           : None
blosc            : None
feather          : None
xlsxwriter       : None
lxml.etree       : 4.4.1
html5lib         : 1.0.1
pymysql          : None
psycopg2         : None
jinja2           : 2.10.3
IPython          : 7.8.0
pandas_datareader: None
bs4              : 4.8.1
bottleneck       : None
fastparquet      : None
gcsfs            : None
lxml.etree       : 4.4.